In [ ]:
#fine tuning aspect based sentiment analysis
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


In [ ]:
# ============================================
# CELL 2: Load MAMS Data from Local Files 
# ============================================

import os

print("=" * 70)
print("📂 LOADING MAMS DATA FROM LOCAL FILES")
print("=" * 70)

# For ACSA (Aspect-Category Sentiment Analysis)
ACSA_TRAIN = "/Users/melindadeng/Measles Research/ABSA/ACSA/train.xml"
ACSA_VAL = "/Users/melindadeng/Measles Research/ABSA/ACSA/val.xml"
ACSA_TEST = "/Users/melindadeng/Measles Research/ABSA/ACSA/test.xml" 

MEASLES_COVID = "/Users/melindadeng/Measles Research/ABSA/ACSA/measles_covid_ACSA_training.csv"

# Splitting measles_covid data into testing, training, and evaluation
from sklearn.model_selection import train_test_split

X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
# test_size=0.2 means 20% of the data will be in the test set.
# random_state ensures reproducibility of the split.

# Step 2: Split main_set into training_set and validation_set
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.25, random_state=42) 
# Here, test_size=0.25 of the *main_set* will be the validation set.
# This results in roughly 60% training, 20% validation, and 20% test (0.8 * 0.75 = 0.6 for train)

# For ATSA (Aspect-Term Sentiment Analysis)
ATSA_TRAIN = "/Users/melindadeng/Measles Research/ABSA/ATSA/train.xml"
ATSA_VAL = "/Users/melindadeng/Measles Research/ABSA/ATSA/val.xml"
ATSA_TEST = "/Users/melindadeng/Measles Research/ABSA/ATSA/test.xml"

# ========================================

# Store paths in a dictionary for easy access
DATA_PATHS = {
    'ACSA': {
        'train': ACSA_TRAIN,
        'val': ACSA_VAL,
        'test': ACSA_TEST
    },
    'ATSA': {
        'train': ATSA_TRAIN,
        'val': ATSA_VAL,
        'test': ATSA_TEST
    }
}


In [ ]:
# ============================================
# CELL 3: Configuration
# ============================================

class Config:
    # Dataset type: 'ATSA' (aspect-term) or 'ACSA' (aspect-category)
    dataset_type = 'ACSA'
    
    # Use the paths from Cell 2
    if dataset_type == 'ACSA':
        data_paths = DATA_PATHS['ACSA']  # ← This line is critical
    else:
        data_paths = DATA_PATHS['ATSA']
    
    # Model: BERTweet (best for social media) or BERT (general)
    use_bertweet = True
    
    if use_bertweet:
        model_name = 'vinai/bertweet-base'
    else:
        model_name = 'bert-base-uncased'
    
    # Training parameters
    max_seq_len = 128
    batch_size = 16
    learning_rate = 2e-5
    num_epochs = 3
    warmup_proportion = 0.1
    dropout = 0.1
    
    # Device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Config()


In [ ]:
# ============================================
# CELL 4: Dataset Class for MAMS
# ============================================
class MAMSDataset(Dataset):
    """Load MAMS dataset from XML files"""
    
    def __init__(self, xml_path, tokenizer, max_len, dataset_type='ACSA'):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.dataset_type = dataset_type
        
        # Parse XML
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Sentiment mapping
        self.sentiment2id = {'positive': 2, 'neutral': 1, 'negative': 0}
        
        # Extract data
        for sentence in root.findall('sentence'):
            text = sentence.find('text').text
            
            if dataset_type == 'ATSA':
                # Aspect-Term Sentiment Analysis
                aspect_terms = sentence.find('aspectTerms')
                if aspect_terms is not None:
                    for aspect_term in aspect_terms.findall('aspectTerm'):
                        aspect = aspect_term.get('term')
                        sentiment = aspect_term.get('polarity')
                        
                        if sentiment in self.sentiment2id:
                            self.data.append({
                                'text': text,
                                'aspect': aspect,
                                'sentiment': sentiment
                            })
            else:
                # Aspect-Category Sentiment Analysis
                aspect_categories = sentence.find('aspectCategories')
                if aspect_categories is not None:
                    for aspect_category in aspect_categories.findall('aspectCategory'):
                        category = aspect_category.get('category')
                        sentiment = aspect_category.get('polarity')
                        
                        if sentiment in self.sentiment2id:
                            self.data.append({
                                'text': text,
                                'aspect': category,
                                'sentiment': sentiment
                            })
        
        print(f"✓ Loaded {len(self.data)} samples from {xml_path}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Tokenize: [CLS] text [SEP] aspect [SEP]
        encoding = self.tokenizer(
            item['text'],
            item['aspect'],
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        sentiment2id = {'positive': 2, 'neutral': 1, 'negative': 0}
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(sentiment2id[item['sentiment']], dtype=torch.long)
        }

In [ ]:
# ============================================
# CELL 5: Model Architecture
# ============================================

class BERTForABSA(nn.Module):
    """BERT/BERTweet model for ABSA"""
    
    def __init__(self, bert_model_name, num_labels=3, dropout=0.1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# ============================================
# CELL 6: Training Functions
# ============================================

def train_epoch(model, dataloader, optimizer, scheduler, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    for batch in progress_bar:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        logits = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(logits, labels)
        
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='macro')
    
    return avg_loss, accuracy, f1

def evaluate(model, dataloader, device):
    """Evaluate the model"""
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(logits, labels)
            
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='macro')
    
    return avg_loss, accuracy, f1, predictions, true_labels


In [ ]:
# ============================================
# CELL 7: Load Data and Create DataLoaders
# ============================================

print("Loading tokenizer:")
if config.use_bertweet:
    tokenizer = AutoTokenizer.from_pretrained(config.model_name, normalization=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)

print("Loading MAMS dataset:")

train_dataset = MAMSDataset(
    config.data_paths['train'], 
    tokenizer,
    config.max_seq_len,
    config.dataset_type
)

val_dataset = MAMSDataset(
    config.data_paths['val'],  
    tokenizer,
    config.max_seq_len,
    config.dataset_type
)

test_dataset = MAMSDataset(
    config.data_paths['test'],  
    tokenizer,
    config.max_seq_len,
    config.dataset_type
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

print(f"\n Data loaded successfully!")

In [ ]:
# ============================================
# CELL 8: Initialize Model and Optimizer
# ============================================

from transformers import get_linear_schedule_with_warmup
print("Imported get_linear_schedule_with_warmup")

device = torch.device(config.device)
print(f"Initializing model on {device}...")

model = BERTForABSA(config.model_name, num_labels=3, dropout=config.dropout).to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=config.learning_rate)
total_steps = len(train_loader) * config.num_epochs
warmup_steps = int(total_steps * config.warmup_proportion)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=warmup_steps, 
    num_training_steps=total_steps
)

print("Model initialized!")
print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {warmup_steps}")

In [ ]:
# ============================================
# CELL 9: Train the Model (Save to specific location)
# ============================================

import os

print(f"\n Training for {config.num_epochs} epochs:")
print("=" * 70)

SAVE_DIR = '/Users/melindadeng/Measles Research/ABSA'  
os.makedirs(SAVE_DIR, exist_ok=True)

best_val_f1 = 0
training_stats = []

for epoch in range(config.num_epochs):
    print(f"\n Epoch {epoch + 1}/{config.num_epochs}")
    print("-" * 70)
    
    train_loss, train_acc, train_f1 = train_epoch(
        model, train_loader, optimizer, scheduler, device
    )
    
    val_loss, val_acc, val_f1, _, _ = evaluate(model, val_loader, device)
    
    print(f"\n Results:")
    print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")
    print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
    
    training_stats.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'train_f1': train_f1,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'val_f1': val_f1
    })
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        
        model_path = os.path.join(SAVE_DIR, 'best_absa_model.pt')
        
        try:
            torch.save({
                'model_state_dict': model.state_dict(),
                'tokenizer_name': config.model_name,
                'dataset_type': config.dataset_type,
                'val_f1': val_f1,
                'epoch': epoch + 1
            }, model_path)
            
            print(f" Saved best model (F1: {best_val_f1:.4f})")
            print(f" Location: {model_path}")
            
        except Exception as e:
            print(f" Error saving model: {e}")

print("Training completed!")

# Show training stats
stats_df = pd.DataFrame(training_stats)
print(" Training Statistics:")
print(stats_df.to_string(index=False))

if os.path.exists(os.path.join(SAVE_DIR, 'best_absa_model.pt')):
    print(f"\n Model saved successfully!")
    print(f" Location: {os.path.join(SAVE_DIR, 'best_absa_model.pt')}")
else:
    print(f"\n Model not saved - please check permissions")

In [ ]:
# ============================================
# CELL 10: Evaluate on Test Set
# ============================================

print("\n Evaluating on test set...")
print("=" * 70)

# Load best model
checkpoint = torch.load('/Users/melindadeng/Measles Research/ABSA/best_absa_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])

test_loss, test_acc, test_f1, test_preds, test_labels = evaluate(
    model, test_loader, device
)

print(f"\n Test Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_acc:.4f}")
print(f"  Macro F1: {test_f1:.4f}")

print(f"\n Detailed Classification Report:")
print(classification_report(
    test_labels, 
    test_preds,
    target_names=['negative', 'neutral', 'positive']
))

In [ ]:
# ============================================
# CELL 11: Load COVID/Measles CSV Data
# ============================================

YOUR_CSV_FILE = '/Users/melindadeng/Measles Research/MergedBluesky_covid_AND_measles.csv'  
TEXT_COLUMN = 'text_clean' 

your_df = pd.read_csv(YOUR_CSV_FILE)


In [ ]:
# ============================================
# CELL 12: Define Aspects to Analyze
# ============================================

# Define aspects you want to analyze sentiment for
aspects = [
    # Vaccine types
    'vaccine',
    'covid vaccine',
    'coronavirus vaccine',
    'measles vaccine',
    'mmr vaccine',
    'mmr',
    'covid',
    'coronavirus',
    'measles',
    'mumps',
    'rubella',
    
     # Vaccine properties
    'side effects',
    'efficacy',
    'effectiveness',
    'safety',
    'protection',
    'immunity',
    
    # Policy/social
    'autism',
    'pregnant',
    'trump',
    'liberals',
    'woke',
    'conservative',
    'republican',
    'government',
    'big pharma',
    'fda',
    'cdc',
    'who'
]

print(f" Will analyze sentiment for {len(aspects)} aspects:")
print("\nVaccine Types:")
print("  " + ", ".join(aspects[:11]))
print("\nVaccine Properties:")
print("  " + ", ".join(aspects[11:17]))
print("\nPolicy/Social:")
print("  " + ", ".join(aspects[17:32]))

In [ ]:
# ============================================
# CELL 13: Apply Model to Data
# ============================================

# Skip if file wasn't loaded successfully
if not os.path.exists(YOUR_CSV_FILE) or TEXT_COLUMN not in your_df.columns:
    print("Skipping analysis - please fix the file path and column name in Cell 11")
else:
    # Load best model
    checkpoint = torch.load('/Users/melindadeng/Measles Research/ABSA/best_absa_model.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"\n Applying ABSA model to your data...")
    print(f"Analyzing {len(your_df)} posts with {len(aspects)} aspects")
    print("This may take a while for large datasets...")
    print("=" * 70)
    
    id2sentiment = {0: 'negative', 1: 'neutral', 2: 'positive'}
    results = []
      
    with torch.no_grad():
        for idx, row in tqdm(your_df.iterrows(), total=len(your_df), desc="Analyzing"):
            text = str(row[TEXT_COLUMN])
            
            # Skip empty or very short texts
            if not text.strip() or len(text) < 10:
                continue
            
            aspect_results = {'text': text}
            
            # Keep original columns from your data
            for col in your_df.columns:
                if col != TEXT_COLUMN:
                    aspect_results[col] = row[col]
            
            # Analyze each aspect
            for aspect in aspects:
                # Tokenize
                encoding = tokenizer(
                    text,
                    aspect,
                    max_length=config.max_seq_len,
                    padding='max_length',
                    truncation=True,
                    return_tensors='pt'
                )
                
                input_ids = encoding['input_ids'].to(device)
                attention_mask = encoding['attention_mask'].to(device)
                
                # Predict
                logits = model(input_ids, attention_mask)
                pred = torch.argmax(logits, dim=1).item()
                sentiment = id2sentiment[pred]
                
                # Get confidence score
                probs = torch.softmax(logits, dim=1)[0]
                confidence = probs[pred].item()
                
                # Store results
                aspect_col = aspect.replace(' ', '_').replace('&', 'and').replace('/', '_')
                aspect_results[f'{aspect_col}_sentiment'] = sentiment
                aspect_results[f'{aspect_col}_confidence'] = round(confidence, 3)
            
            results.append(aspect_results)
    
    print(f"\n Analysis complete!")
    print(f"Analyzed {len(results)} posts")

In [ ]:
# ============================================
# CELL 14: Create Results DataFrame
# ============================================

if 'results' in locals() and len(results) > 0:
    results_df = pd.DataFrame(results)
    
    print(f"\n Results Summary:")
    print(f"  Total analyzed: {len(results_df)} posts")
    print(f"  Total columns: {len(results_df.columns)}")
    
    # Show column names
    sentiment_cols = [col for col in results_df.columns if col.endswith('_sentiment')]
    print(f"  Sentiment columns: {len(sentiment_cols)}")
    
    # Show first few results
    print(f"\n First 3 results (text + first 3 aspects):")
    display_cols = ['text'] + sentiment_cols[:3]
    print(results_df[display_cols].head(3).to_string())
else:
    print("No results to display")

In [ ]:
# ============================================
# CELL 15: Analyze Sentiment Distribution
# ============================================

if 'results_df' in locals():
    print("\n Sentiment Distribution by Aspect:")
    print("=" * 70)
    
    sentiment_cols = [col for col in results_df.columns if col.endswith('_sentiment')]
    
    # Create summary for all aspects
    summary_data = []
    
    for col in sentiment_cols:
        aspect_name = col.replace('_sentiment', '').replace('_', ' ')
        counts = results_df[col].value_counts()
        
        total = len(results_df)
        pos = counts.get('positive', 0)
        neu = counts.get('neutral', 0)
        neg = counts.get('negative', 0)        
        summary_data.append({
            'aspect': aspect_name,
            'positive': pos,
            'positive_pct': f"{(pos/total*100):.1f}%",
            'neutral': neu,
            'neutral_pct': f"{(neu/total*100):.1f}%",
            'negative': neg,
            'negative_pct': f"{(neg/total*100):.1f}%"
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    print("\n Summary Table:")
    print(summary_df.to_string(index=False))
    
    # Show detailed view for top 5 aspects
    print("\n\n Detailed Distribution (Top 5 Aspects):")
    print("=" * 70)
    
    for col in sentiment_cols[:5]:
        aspect_name = col.replace('_sentiment', '').replace('_', ' ')
        print(f"\n{aspect_name.upper()}:")
        
        counts = results_df[col].value_counts()
        for sentiment, count in counts.items():
            pct = (count / len(results_df)) * 100
            bar = '█' * int(pct / 2)
            print(f"  {sentiment:8s}: {count:4d} ({pct:5.1f}%) {bar}")
else:
    print(" No results to analyze")


In [ ]:
# ============================================
# CELL 16: Save Results
# ============================================

if 'results_df' in locals():
    # Save full results
    output_file = '/Users/melindadeng/Measles Research/ABSA/absa_results.csv'
    results_df.to_csv(output_file, index=False)
    print(f" Full results saved to: {output_file}")
    
    # Save summary statistics
    if 'summary_df' in locals():
        summary_file = '/Users/melindadeng/Measles Research/ABSA/absa_summary.csv'
        summary_df.to_csv(summary_file, index=False)
        print(f" Summary statistics saved to: {summary_file}")
    
    print(f"\n All results saved successfully!")
    print(f"\n Output files:")
    print(f"  1. {output_file} - Full results with all aspects and confidences")
    print(f"  2. {summary_file} - Summary statistics by aspect")
    print(f"  3. best_absa_model.pt - Trained model (can be reused)")
else:
    print(" No results to save")

ATSA---------------------------------------------------------

In [ ]:
# ============================================
# CELL 1: Install and Import Dependencies
# ============================================

#!pip install torch transformers pandas scikit-learn tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import (
    AutoTokenizer, 
    AutoModel,  
    get_linear_schedule_with_warmup
)
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print(f" PyTorch version: {torch.__version__}")
print(f" CUDA available: {torch.cuda.is_available()}")

In [ ]:

# ============================================
# CELL 2: Load MAMS-ATSA Data from Local Files
# ============================================

import os

print("=" * 70)
print(" LOADING MAMS-ATSA DATA FROM LOCAL FILES")
print("=" * 70)

# ========================================
# PASTE YOUR FILE PATHS HERE FOR ATSA
# ========================================

# For ATSA (Aspect-Term Sentiment Analysis)
ATSA_TRAIN = "/Users/melindadeng/Measles Research/ABSA/ATSA/train.xml"
ATSA_VAL = "/Users/melindadeng/Measles Research/ABSA/ATSA/val.xml"
ATSA_TEST = "/Users/melindadeng/Measles Research/ABSA/ATSA/test.xml"

# ========================================

# Store paths in dictionary
DATA_PATHS = {
    'train': ATSA_TRAIN,
    'val': ATSA_VAL,
    'test': ATSA_TEST
}

# Verify all files exist
print("\n Verifying files...")
print("-" * 70)

all_good = True
for split, path in DATA_PATHS.items():
    if os.path.exists(path):
        size = os.path.getsize(path)
        print(f"  {split}.xml ({size:,} bytes)")
    else:
        print(f"  {split}.xml - NOT FOUND: {path}")
        all_good = False


# ============================================
# CELL 3: Configuration
# ============================================

class Config:
    # Dataset type
    dataset_type = 'ATSA'  # Aspect-Term Sentiment Analysis
    
    # Data paths from Cell 2
    data_paths = DATA_PATHS
    
    # Model: BERTweet (best for social media) or BERT (general)
    use_bertweet = True
    
    if use_bertweet:
        model_name = 'vinai/bertweet-base'
    else:
        model_name = 'bert-base-uncased'
    
    # Training parameters
    max_seq_len = 128
    batch_size = 16
    learning_rate = 2e-5
    num_epochs = 3
    warmup_proportion = 0.1
    dropout = 0.1
    
    # Device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Config()


# ============================================
# CELL 4: Dataset Class for MAMS-ATSA
# ============================================

class ATSADataset(Dataset):
    """Load MAMS-ATSA dataset from XML files"""
    
    def __init__(self, xml_path, tokenizer, max_len):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        # Parse XML
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Sentiment mapping
        self.sentiment2id = {'positive': 2, 'neutral': 1, 'negative': 0}
        
        # Extract data - ATSA format
        for sentence in root.findall('sentence'):
            text = sentence.find('text').text
            
            # Get aspect terms (this is ATSA-specific)
            aspect_terms = sentence.find('aspectTerms')
            if aspect_terms is not None:
                for aspect_term in aspect_terms.findall('aspectTerm'):
                    term = aspect_term.get('term')
                    sentiment = aspect_term.get('polarity')
                    
                    if sentiment in self.sentiment2id:
                        self.data.append({
                            'text': text,
                            'aspect': term,  # The specific term extracted from text
                            'sentiment': sentiment
                        })
        
        print(f"✓ Loaded {len(self.data)} aspect-term pairs from {xml_path}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Tokenize: [CLS] text [SEP] aspect_term [SEP]
        encoding = self.tokenizer(
            item['text'],
            item['aspect'],
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(
                self.sentiment2id[item['sentiment']], 
                dtype=torch.long
            ),
            'text': item['text'],
            'aspect': item['aspect']
        }

In [ ]:
# ============================================
# CELL 5: Model Architecture
# ============================================

class BERTForATSA(nn.Module):
    """BERT/BERTweet model for ATSA"""
    
    def __init__(self, bert_model_name, num_labels=3, dropout=0.1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # Use [CLS] token representation
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# ============================================
# CELL 6: Training Functions
# ============================================

def train_epoch(model, dataloader, optimizer, scheduler, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    for batch in progress_bar:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        logits = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(logits, labels)
        
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='macro')
    
    return avg_loss, accuracy, f1

def evaluate(model, dataloader, device):
    """Evaluate the model"""
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(logits, labels)
            
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='macro')
    
    return avg_loss, accuracy, f1, predictions, true_labels

# ============================================
# CELL 7: Load Data and Create DataLoaders
# ============================================

if config.use_bertweet:
    tokenizer = AutoTokenizer.from_pretrained(config.model_name, normalization=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)


# Use direct paths from config
train_dataset = ATSADataset(
    config.data_paths['train'],
    tokenizer,
    config.max_seq_len
)

val_dataset = ATSADataset(
    config.data_paths['val'],
    tokenizer,
    config.max_seq_len
)

test_dataset = ATSADataset(
    config.data_paths['test'],
    tokenizer,
    config.max_seq_len
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

# ============================================
# CELL 8: Initialize Model and Optimizer
# ============================================

device = torch.device(config.device)

model = BERTForATSA(config.model_name, num_labels=3, dropout=config.dropout).to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=config.learning_rate)
total_steps = len(train_loader) * config.num_epochs
warmup_steps = int(total_steps * config.warmup_proportion)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=warmup_steps, 
    num_training_steps=total_steps
)


print(f" Total training steps: {total_steps}")
print(f" Warmup steps: {warmup_steps}")

In [ ]:
# ============================================
# CELL 9: Train the Model
# ============================================

import os


# Set save directory
SAVE_DIR = '/Users/melindadeng/Measles Research/ABSA'
os.makedirs(SAVE_DIR, exist_ok=True)

best_val_f1 = 0
training_stats = []

for epoch in range(config.num_epochs):
    print(f"\n Epoch {epoch + 1}/{config.num_epochs}")
    print("-" * 70)
    
    train_loss, train_acc, train_f1 = train_epoch(
        model, train_loader, optimizer, scheduler, device
    )
    
    val_loss, val_acc, val_f1, _, _ = evaluate(model, val_loader, device)
    
    print(f"\n Results:")
    print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")
    print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
    
    training_stats.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'train_f1': train_f1,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'val_f1': val_f1
    })
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        
        # Save to specific directory
        model_path = os.path.join(SAVE_DIR, 'best_atsa_model.pt')
        
        try:
            torch.save({
                'model_state_dict': model.state_dict(),
                'tokenizer_name': config.model_name,
                'dataset_type': config.dataset_type,
                'val_f1': val_f1,
                'epoch': epoch + 1
            }, model_path)
            
            print(f"   Saved best model (F1: {best_val_f1:.4f})")
            print(f"     Location: {model_path}")
            
        except Exception as e:
            print(f"   Error saving model: {e}")


# Show training stats
stats_df = pd.DataFrame(training_stats)
print("\n Training Statistics:")
print(stats_df.to_string(index=False))

In [ ]:
# ============================================
# CELL 10: Evaluate on Test Set
# ============================================


# Load best model
model_path = os.path.join(SAVE_DIR, 'best_atsa_model.pt')
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

test_loss, test_acc, test_f1, test_preds, test_labels = evaluate(
    model, test_loader, device
)

print(f"\n Test Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_acc:.4f}")
print(f"  Macro F1: {test_f1:.4f}")

print(f"\n Detailed Classification Report:")
print(classification_report(
    test_labels, 
    test_preds,
    target_names=['negative', 'neutral', 'positive']
))

In [ ]:
# ============================================
# CELL 11: ATSA Aspect Extraction Helper
# ============================================

import re

def extract_potential_aspects(text, common_aspects=None):
    """
    Extract potential aspect terms from text
    """
    
    # Common vaccine-related terms
    if common_aspects is None:
        common_aspects = [
            # Vaccine types/brands
            'vaccine', 'vaccination', 'mmr', 'measles', 'covid', 'coronavirus',
            
            # Effects/Properties
            'side effects', 'adverse effects', 'reaction', 
            'efficacy', 'effectiveness', 'protection', 'immunity',
            'safety', 'risk', 'danger', 'death', 'pregnant'
        ]
    
    text_lower = text.lower()
    found_aspects = []
    
    # Check for each aspect (longest first to catch phrases)
    sorted_aspects = sorted(common_aspects, key=len, reverse=True)
    
    for aspect in sorted_aspects:
        # Use word boundaries to match whole words/phrases
        pattern = r'\b' + re.escape(aspect) + r'\b'
        if re.search(pattern, text_lower):
            found_aspects.append(aspect)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_aspects = []
    for aspect in found_aspects:
        if aspect not in seen:
            seen.add(aspect)
            unique_aspects.append(aspect)
    
    return unique_aspects

# Test the function
test_texts = [
    "Pfizer vaccine is great but side effects are terrible",
    "Got my COVID shot today",
    "Vaccine mandates infringe on freedom"
]

print(" Testing aspect extraction:")
print("=" * 70)
for text in test_texts:
    aspects = extract_potential_aspects(text)
    print(f"\nText: {text}")
    print(f"Extracted aspects: {aspects}")

In [ ]:
# ============================================
# CELL 12: Load COVID/Measles Data
# ============================================

YOUR_CSV_FILE = '/Users/melindadeng/Measles Research/MergedBluesky_covid_AND_measles.csv'  
TEXT_COLUMN = 'text_clean' 

# ========================================

if not os.path.exists(YOUR_CSV_FILE):
    print(f"\n File not found: {YOUR_CSV_FILE}")
else:
    # Load your data
    your_df = pd.read_csv(YOUR_CSV_FILE)
    
    print(f"Loaded {len(your_df)} rows")
    print(f"Columns: {list(your_df.columns)}")
    
    if TEXT_COLUMN not in your_df.columns:
        print(f"\n Column '{TEXT_COLUMN}' not found!")
        print(f"Available columns: {list(your_df.columns)}")
    else:
        print(f"\n First few rows of '{TEXT_COLUMN}':")
        for i, text in enumerate(your_df[TEXT_COLUMN].head(3), 1):
            print(f"\n{i}. {text[:150]}...")
            

In [ ]:
# ============================================
# CELL 13: Apply ATSA Model to Your Data
# ============================================

if not os.path.exists(YOUR_CSV_FILE) or TEXT_COLUMN not in your_df.columns:
    print("Skipping analysis")
else:
    # Load best model
    model_path = os.path.join(SAVE_DIR, 'best_atsa_model.pt')
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    id2sentiment = {0: 'negative', 1: 'neutral', 2: 'positive'}
    results = []
    
    with torch.no_grad():
        for idx, row in tqdm(your_df.iterrows(), total=len(your_df), desc="Analyzing"):
            text = str(row[TEXT_COLUMN])
            
            # Skip empty or very short texts
            if not text.strip() or len(text) < 10:
                continue
            
            # Extract potential aspects from this text
            extracted_aspects = extract_potential_aspects(text)
            
            # If no aspects found, skip
            if not extracted_aspects:
                continue
            
            # Analyze sentiment for each extracted aspect
            aspect_sentiments = []
            
            for aspect in extracted_aspects:
                # Tokenize
                encoding = tokenizer(
                    text,
                    aspect,
                    max_length=config.max_seq_len,
                    padding='max_length',
                    truncation=True,
                    return_tensors='pt'
                )
                
                input_ids = encoding['input_ids'].to(device)
                attention_mask = encoding['attention_mask'].to(device)
                
                # Predict
                logits = model(input_ids, attention_mask)
                pred = torch.argmax(logits, dim=1).item()
                sentiment = id2sentiment[pred]
                
                # Get confidence
                probs = torch.softmax(logits, dim=1)[0]
                confidence = probs[pred].item()
                
                aspect_sentiments.append({
                    'aspect': aspect,
                    'sentiment': sentiment,
                    'confidence': round(confidence, 3)
                })
            
            # Store results
            result = {
                'text': text,
                'extracted_aspects': [a['aspect'] for a in aspect_sentiments],
                'aspect_sentiments': aspect_sentiments
            }
            
            # Keep original columns
            for col in your_df.columns:
                if col != TEXT_COLUMN:
                    result[col] = row[col]
            
            results.append(result)
    
    print(f"\n Analysis complete!")
    print(f"Analyzed {len(results)} posts with extracted aspects")


In [ ]:
# ============================================
# CELL 14: Format ATSA Results
# ============================================

if 'results' in locals() and len(results) > 0:
    
    # Create a detailed results dataframe
    detailed_results = []
    
    for result in results:
        base_info = {
            'text': result['text'],
            'num_aspects': len(result['extracted_aspects']),
            'aspects': ', '.join(result['extracted_aspects'])
        }
        
        # Add each aspect-sentiment pair
        for i, asp_sent in enumerate(result['aspect_sentiments'], 1):
            detailed_results.append({
                **base_info,
                'aspect_term': asp_sent['aspect'],
                'sentiment': asp_sent['sentiment'],
                'confidence': asp_sent['confidence']
            })
    
    results_df = pd.DataFrame(detailed_results)
    
    print(f"\n Results Summary:")
    print(f"  Posts analyzed: {len(results)}")
    print(f"  Total aspect-sentiment pairs: {len(results_df)}")
    print(f"  Unique aspects found: {results_df['aspect_term'].nunique()}")
    
    # Show most common aspects
    print(f"\n Most Common Extracted Aspects:")
    top_aspects = results_df['aspect_term'].value_counts().head(10)
    for aspect, count in top_aspects.items():
        print(f"  {aspect}: {count} mentions")
    
    # Show first few results
    print(f"\n Sample results:")
    print(results_df[['text', 'aspect_term', 'sentiment', 'confidence']].head(10).to_string())

else:
    print("No results to display")



In [ ]:
# ============================================
# CELL 15: Analyze ATSA Results
# ============================================

if 'results_df' in locals():
    print("\n Sentiment Distribution by Aspect:")
    print("=" * 70)
    
    # Get top 10 most common aspects
    top_aspects = results_df['aspect_term'].value_counts().head(10).index
    
    for aspect in top_aspects:
        aspect_data = results_df[results_df['aspect_term'] == aspect]
        
        print(f"\n{aspect.upper()} ({len(aspect_data)} mentions):")
        
        sentiment_counts = aspect_data['sentiment'].value_counts()
        for sentiment, count in sentiment_counts.items():
            pct = (count / len(aspect_data)) * 100
            bar = '█' * int(pct / 5)
            print(f"  {sentiment:8s}: {count:3d} ({pct:5.1f}%) {bar}")
        
        # Average confidence
        avg_conf = aspect_data['confidence'].mean()
        print(f"  Avg confidence: {avg_conf:.3f}")



In [ ]:
# ============================================
# CELL 16: Save ATSA Results
# ============================================

if 'results_df' in locals():
    # Save detailed results
    output_file = os.path.join(SAVE_DIR, 'atsa_results_detailed.csv')
    results_df.to_csv(output_file, index=False)
    
    # Create summary by post
    summary_data = []
    for result in results:
        summary = {
            'text': result['text'],
            'num_aspects_found': len(result['extracted_aspects']),
            'aspects_list': ', '.join(result['extracted_aspects'])
        }
        
        # Add sentiment counts
        sentiments = [a['sentiment'] for a in result['aspect_sentiments']]
        summary['num_positive'] = sentiments.count('positive')
        summary['num_neutral'] = sentiments.count('neutral')
        summary['num_negative'] = sentiments.count('negative')
        
        # Overall sentiment (majority vote)
        if sentiments:
            from collections import Counter
            most_common = Counter(sentiments).most_common(1)[0][0]
            summary['overall_sentiment'] = most_common
        
        summary_data.append(summary)
    
    summary_df = pd.DataFrame(summary_data)
    summary_file = os.path.join(SAVE_DIR, 'atsa_results_summary.csv')
    summary_df.to_csv(summary_file, index=False)
    print(f" Summary saved to: {summary_file}")




In [ ]:
# ============================================
# CELL 17: Compare Specific Aspects
# ============================================

if 'results_df' in locals():
    
    # Define aspects you want to compare
    aspects_to_compare = ['mmr', 'covid', 'vaccine']
    
    print("\n Comparing Specific Aspects:")
    print("=" * 70)
    
    comparison_data = []
    
    for aspect in aspects_to_compare:
        aspect_data = results_df[results_df['aspect_term'] == aspect]
        
        if len(aspect_data) > 0:
            comparison_data.append({
                'aspect': aspect,
                'mentions': len(aspect_data),
                'positive': (aspect_data['sentiment'] == 'positive').sum(),
                'neutral': (aspect_data['sentiment'] == 'neutral').sum(),
                'negative': (aspect_data['sentiment'] == 'negative').sum(),
                'avg_confidence': aspect_data['confidence'].mean()
            })
    
    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        print("\n" + comparison_df.to_string(index=False))
        
        # Calculate percentages
        comparison_df['pct_positive'] = (comparison_df['positive'] / comparison_df['mentions'] * 100).round(1)
        comparison_df['pct_negative'] = (comparison_df['negative'] / comparison_df['mentions'] * 100).round(1)
        
        print("\n Percentage View:")
        print(comparison_df[['aspect', 'mentions', 'pct_positive', 'pct_negative']].to_string(index=False))



In [ ]:
# ============================================
# CELL 18: Example - Find Posts with Multiple Aspects
# ============================================

if 'results' in locals():
    
    print("\n Posts with Multiple Aspects:")
    print("=" * 70)
    
    # Find posts with 3+ aspects
    multi_aspect_posts = [r for r in results if len(r['extracted_aspects']) >= 3]
    
    print(f"\nFound {len(multi_aspect_posts)} posts with 3+ aspects")
    
    # Show examples
    for i, result in enumerate(multi_aspect_posts[:5], 1):
        print(f"\n{i}. Text: {result['text'][:200]}...")
        print(f"   Aspects found: {', '.join(result['extracted_aspects'])}")
        print(f"   Sentiments:")
        for asp_sent in result['aspect_sentiments']:
            print(f"      {asp_sent['aspect']} → {asp_sent['sentiment']} (conf: {asp_sent['confidence']:.3f})")